In [1]:
import custom_environments
import gymnasium as gym
import os
import pandas as pd
import numpy as np

In [2]:
machines_data = []

print ("Opening files machine data files:")
path_to_machine_data = os.path.join(os.getcwd(), "machine_data")
for filename in os.listdir(path_to_machine_data):
  df = pd.read_csv(os.path.join(path_to_machine_data, filename))
  machine_name = os.path.splitext(filename)[0]
  print(machine_name)
  print(df.head())
  machine_data = [np.float32(row[1]) for row in df.values.tolist()]
  machines_data.append(machine_data)

Opening files machine data files:
m_1932
   time_step  cpu_util_percent
0          0         36.444444
1          1         33.750000
2          2         31.250000
3          3         35.600000
4          4         34.800000
m_1933
   time_step  cpu_util_percent
0          0          5.240000
1          1          5.000000
2          2          5.470588
3          3          5.923077
4          4          5.153846
m_1934
   time_step  cpu_util_percent
0          0         46.800000
1          1         43.857143
2          2         52.000000
3          3         54.500000
4          4         39.750000
m_1935
   time_step  cpu_util_percent
0          0         40.863636
1          1         39.500000
2          2         37.833333
3          3         38.875000
4          4         38.000000
m_1936
   time_step  cpu_util_percent
0          0         38.200000
1          1         39.636364
2          2         33.000000
3          3         31.500000
4          4         32.750000
m

In [3]:
num_datacentres = 2
datacentre_mapping = {
  0: 0, 1: 0, 2: 0, 3: 0, 4: 0,
  5: 1, 6: 1, 7: 1, 8: 1, 9: 1,
}

In [4]:
env = gym.make('custom_environments/MultiDataCenterEnvironment', 
                machines_data=machines_data, 
                datacentre_mapping=datacentre_mapping, 
                num_datacentres=num_datacentres
              )

In [5]:
terminated = False
done = False
env.reset()
while not (done or terminated):
  observation, reward, done, terminated, info = env.step(10)
  print(observation)



{'machines_curr_state': (array([33.75], dtype=float32), array([5.], dtype=float32), array([43.857143], dtype=float32), array([39.5], dtype=float32), array([39.636364], dtype=float32), array([45.], dtype=float32), array([29.], dtype=float32), array([49.], dtype=float32), array([45.], dtype=float32), array([24.], dtype=float32)), 'workload': array([96.42313], dtype=float32), 'workload_datacentre': 1}
{'machines_curr_state': (array([31.25], dtype=float32), array([5.470588], dtype=float32), array([52.], dtype=float32), array([37.833332], dtype=float32), array([33.], dtype=float32), array([36.333332], dtype=float32), array([35.2], dtype=float32), array([47.35294], dtype=float32), array([41.], dtype=float32), array([24.6], dtype=float32)), 'workload': array([38.73855], dtype=float32), 'workload_datacentre': 0}
{'machines_curr_state': (array([35.6], dtype=float32), array([5.923077], dtype=float32), array([54.5], dtype=float32), array([38.875], dtype=float32), array([31.5], dtype=float32), arr